In [1]:
from typing import Dict, List, Optional
from collections import Counter
import os
import csv
!pip install torchmetrics
!pip install pytorch-metric-learning
import urllib.request
from torch.utils.data import DataLoader
!pip install pytorch-lightning
import torch.optim as optim
import torchmetrics
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
url = 'https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py'
filename = 'conlleval.py'
urllib.request.urlretrieve(url, filename)
from conlleval import evaluate


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
class Tokenizer:
    def __init__(self):
        # two special tokens for padding and unknown
        self.token2idx = {"<pad>": 0, "<unk>": 1}
        self.idx2token = ["<pad>", "<unk>"]
        self.is_fit = False
    
    @property
    def pad_id(self):
        return self.token2idx["<pad>"]
    
    def __len__(self):
        return len(self.idx2token)
    
    def fit(self, train_texts: List[str]):
        counter = Counter()
        for text in train_texts:
            counter.update(text.lower().split())
        
        # manually set a vocabulary size for the data set
        vocab_size = 20000
        self.idx2token.extend([token for token, count in counter.most_common(vocab_size - 2)])
        for (i, token) in enumerate(self.idx2token):
            self.token2idx[token] = i
            
        self.is_fit = True
                
    def encode(self, text: str, max_length: Optional[int] = None) -> List[int]:
        if not self.is_fit:
            raise Exception("Please fit the tokenizer on the training tokens")
            
        # Split the text into tokens and encode each token using the token2idx mapping
        tokens = text.lower().split()
        token_ids = [self.token2idx.get(token, self.token2idx["<unk>"]) for token in tokens]

        # Pad or truncate the token ids based on the max_length parameter
        if max_length is not None:
            if len(token_ids) < max_length:
                token_ids += [self.token2idx["<pad>"]] * (max_length - len(token_ids))
            else:
                token_ids = token_ids[:max_length]

        return token_ids


In [3]:
def load_raw_data(filepath: str, with_tags: bool = True):
    data = {'text': []}
    if with_tags:
        data['tags'] = []
        with open(filepath) as f:
            reader = csv.reader(f)
            for text, tags in reader:
                data['text'].append(text)
                data['tags'].append(tags)
    else:
        with open(filepath) as f:
            for line in f:
                data['text'].append(line.strip())
    return data

In [4]:
tokenizer = Tokenizer()
data_dir = os.getcwd()
train_raw = load_raw_data(os.path.join(data_dir, "train.csv"))
val_raw = load_raw_data(os.path.join(data_dir, "val.csv"))
test_raw = load_raw_data(os.path.join(data_dir, "test_tokens.txt"), with_tags=False)
# fit the tokenizer on the training tokens
tokenizer.fit(train_raw['text'])

In [5]:
newtext = 'how are you doing today ?'
tokenizer.encode(newtext, max_length=10)


[803, 57, 256, 1313, 755, 1813, 0, 0, 0, 0]

In [6]:
print(len(train_raw['text']))

14041


In [7]:
#upload the dataset
#for google colb, use this
#from google.colab import files
#uploaded = files.upload()

In [8]:
class NERDataset: 
    tag2idx = {'O': 1, 'B-PER': 2, 'I-PER': 3, 'B-ORG': 4, 'I-ORG': 5, 'B-LOC': 6, 'I-LOC': 7, 'B-MISC': 8, 'I-MISC': 9}
    idx2tag = ['<pad>', 'O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG','B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
  
    def __init__(self, raw_data: Dict[str, List[str]], tokenizer: Tokenizer, max_length: int = 128):
        self.tokenizer = tokenizer
        self.token_ids = []
        self.tag_ids = []
        self.with_tags = False
        for text in raw_data['text']:
            self.token_ids.append(tokenizer.encode(text, max_length=max_length))
        if 'tags' in raw_data:
            self.with_tags = True
            for tags in raw_data['tags']:
                self.tag_ids.append(self.encode_tags(tags, max_length=max_length))

    
    def encode_tags(self, tags: str, max_length: Optional[int] = None):
        tag_ids = [self.tag2idx[tag] for tag in tags.split()]
        if max_length is None:
            return tag_ids
        # truncate the tags if longer than max_length
        if len(tag_ids) > max_length:
            return tag_ids[:max_length]
        # pad with 0s if shorter than max_length
        else:
            return tag_ids + [0] * (max_length - len(tag_ids))  # 0 as padding for tags
        
    def __len__(self):
        return len(self.token_ids)
    
    def __getitem__(self, idx):
        token_ids = torch.LongTensor(self.token_ids[idx])
        mask = token_ids == self.tokenizer.pad_id  # padding tokens
        if self.with_tags:
            # for training and validation
            return token_ids, mask, torch.LongTensor(self.tag_ids[idx])
        else:
            # for testing
            return token_ids, mask
        

In [9]:
tr_data = NERDataset(train_raw, tokenizer)
va_data = NERDataset(val_raw, tokenizer)
te_data = NERDataset(test_raw, tokenizer)

In [10]:
samplenum = 265
print(len(tr_data))
print(train_raw['text'][samplenum])
print(tr_data[samplenum])


14041
The Greek socialist party 's executive bureau gave Prime Minister Costas Simitis its backing if he chooses to call snap elections , its general secretary Costas Skandalidis told reporters on Thursday .
(tensor([    2,  1638,  1466,   147,    15,   996,  1877,   407,   229,   103,
         2415,  2672,    63,  2698,   141,    26, 11142,     7,   629,  2673,
          269,     4,    63,   335,   750,  2415,  4652,    90,   524,    13,
           70,     3,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           

In [11]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, hidden_size, num_layers):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        encoder_layer = nn.TransformerEncoderLayer(embed_size, num_heads, hidden_size)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.linear = nn.Linear(embed_size, 10)

    def forward(self, src, src_mask):
        # src shape: (batch_size, max_length)
        # src_mask shape: (batch_size, max_length)
        embedded = self.embedding(src)  # shape: (batch_size, max_length, embed_size)
        encoded = self.transformer_encoder(embedded.transpose(0, 1), src_key_padding_mask=src_mask).transpose(0, 1)  # shape: (batch_size, max_length, embed_size)
        output = self.linear(encoded)  # shape: (batch_size, max_length, num_classes)
        return output

In [19]:
# def init_weights(module):
#     if isinstance(module, (nn.Linear, nn.Embedding)):
#         nn.init.xavier_uniform_(module.weight)

In [51]:
def init_weights(module):
    if isinstance(module, (nn.Linear, nn.Embedding)):
        nn.init.kaiming_uniform_(module.weight, nonlinearity='relu')

In [40]:
def validate(
    model: nn.Module, 
    dataloader: DataLoader, 
    device: torch.device,
):
    acc_metric = torchmetrics.Accuracy(task = 'multiclass', num_classes = 10, compute_on_step=False).to(device)
    loss_metric = torchmetrics.MeanMetric(compute_on_step=False).to(device)
    model.eval()
    
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids, input_mask, tags = batch[0].to(device), batch[1].to(device), batch[2].to(device)
            # output shape: (batch_size, max_length, num_classes)
            logits = model(input_ids, input_mask)
            # ignore padding index 0 when calculating loss
            loss = F.cross_entropy(logits.reshape(-1, 10), tags.reshape(-1), ignore_index=0)
                
            loss_metric.update(loss, input_mask.numel() - input_mask.sum())
            is_active = torch.logical_not(input_mask)  # non-padding elements
            # only consider non-padded tokens when calculating accuracy
            acc_metric.update(logits[is_active], tags[is_active])

    print(f"| Epoch {epoch+1} | Validate | loss {loss_metric.compute():.6f} | acc {acc_metric.compute():.6f} |!!!!!!!!!!!!!!!!!!!!!!!!!")

    return loss_metric.compute()


In [41]:
#modify as required
def train(
    model: nn.Module, 
    dataloader: DataLoader, 
    optimizer: optim.Optimizer,
    device: torch.device,
    epoch: int,
):
    acc_metric = torchmetrics.Accuracy(task = 'multiclass', num_classes = 10, compute_on_step=False).to(device)
    loss_metric = torchmetrics.MeanMetric(compute_on_step=False).to(device)
    model.train()


    # loop through all batches in the training
    for batch in tqdm(dataloader):
        input_ids, input_mask, tags = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        optimizer.zero_grad()
        # output shape: (batch_size, max_length, num_classes)
        logits = model(input_ids, input_mask)
        # ignore padding index 0 when calculating loss
        loss = F.cross_entropy(logits.reshape(-1, 10), tags.reshape(-1), ignore_index=0)
        
        loss.backward()
        optimizer.step()
        
        loss_metric.update(loss, input_mask.numel() - input_mask.sum())
        is_active = torch.logical_not(input_mask)  # non-padding elements
        # only consider non-padded tokens when calculating accuracy
        acc_metric.update(logits[is_active], tags[is_active])
    
    print(f"| Epoch {epoch+1} | Train | loss {loss_metric.compute():.6f} | acc {acc_metric.compute():.6f} |")


In [14]:
def predict(model: nn.Module, dataloader: DataLoader, device: torch.device) -> List[List[str]]:
    model.eval()
    preds = []

    idx2tag = ['<pad>', 'O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids, src_mask = batch[0].to(device), batch[1].to(device)
            logits = model(input_ids, src_mask=src_mask)
            batch_preds = logits.argmax(dim=-1).tolist()
            for i, tags in enumerate(batch_preds):
                # Get the true length of the unpadded sequence using the input mask
                seq_len = 128 - src_mask[i].sum().item()  # .item() converts a one-element tensor to a scalar
                # print(src_mask[i])
                # print(seq_len)
                # Convert the predicted tag indices to tag labels
                tag_preds = [idx2tag[idx] for idx in tags[:seq_len]]  # Slice the predicted tags to seq_len
                # print(tag_preds)
                # Append the tag predictions to the list of predictions
                preds.append(tag_preds)

    return preds

In [49]:
import optuna
def objective(trial: optuna.Trial):
    # Suggest hyperparameters
    embed_size =  trial.suggest_categorical("embed_size", [128, 256, 512])
    num_heads = trial.suggest_categorical("num_heads", [2, 4, 8])
    hidden_size = trial.suggest_categorical("hidden_size", [128, 256, 512])
    num_layers = trial.suggest_int("num_layers", 1, 4)
    lr = trial.suggest_float("lr", 5e-5, 1e-3)

    # Create the model with the suggested hyperparameters
    model = TransformerModel(vocab_size=len(tokenizer),
                             embed_size=embed_size,
                             num_heads=num_heads,
                             hidden_size=hidden_size,
                             num_layers=num_layers).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.apply(init_weights)

    best_val_loss = float('inf')
    best_epoch = 0
    best_model_state = None

    for epoch in range(max_epochs):
        train(model, train_dataloader, optimizer, device, epoch)

        val_loss = validate(model, val_dataloader, device)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            best_model_state = model.state_dict()

            print(f"| Epoch {epoch+1} | New best validation loss: {best_val_loss:.4f} **********************")
        elif epoch - best_epoch >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}.")
            print(f"Best epoch is: {best_epoch + 1}")
            break

    # Restore the best model state
    model.load_state_dict(best_model_state)

    return best_val_loss

In [52]:
torch.manual_seed(321)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

#hyperparameters
BATCH_SIZE = 32
max_epochs = 10
patience = 2

# data loaders
train_dataloader = DataLoader(tr_data, batch_size = BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(va_data, batch_size = BATCH_SIZE)
test_dataloader = DataLoader(te_data, batch_size = BATCH_SIZE)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value:.4f}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2023-03-16 19:11:20,840] A new study created in memory with name: no-name-0831b67d-f29b-4716-8eb6-ba12d537c003


cuda


100%|██████████| 439/439 [00:08<00:00, 54.21it/s]


| Epoch 1 | Train | loss 0.405201 | acc 0.892791 |


100%|██████████| 102/102 [00:00<00:00, 164.84it/s]


...

| Epoch 4 | Validate | loss 0.269891 | acc 0.931077 |!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 439/439 [00:07<00:00, 57.04it/s]


| Epoch 4 | Train | loss 0.080585 | acc 0.973736 |


100%|██████████| 102/102 [00:00<00:00, 160.12it/s]
[I 2023-03-16 19:41:38,520] Trial 29 finished with value: 0.26310837268829346 and parameters: {'embed_size': 256, 'num_heads': 8, 'hidden_size': 512, 'num_layers': 2, 'lr': 0.00039022726015531454}. Best is trial 15 with value: 0.23818016052246094.


| Epoch 4 | Validate | loss 0.308337 | acc 0.931934 |!!!!!!!!!!!!!!!!!!!!!!!!!
Early stopping triggered at epoch 4.
Best epoch is: 2
Best trial:
  Value: 0.2382
  Params: 
    embed_size: 256
    num_heads: 4
    hidden_size: 256
    num_layers: 3
    lr: 0.000456008621520148


In [28]:
prediction = predict(model, val_dataloader, device)
pred_tags = []
for tags in prediction:
    pred_tags.extend(tags)
    pred_tags.append('O')

true_tags = []
for tags in val_raw['tags']:
    true_tags.extend(tags.strip().split())
    true_tags.append('O')

evaluate(true_tags, pred_tags)

processed 54612 tokens with 5942 phrases; found: 5956 phrases; correct: 4140.
accuracy:  67.53%; (non-O)
accuracy:  93.80%; precision:  69.51%; recall:  69.67%; FB1:  69.59
              LOC: precision:  83.95%; recall:  80.89%; FB1:  82.40  1770
             MISC: precision:  76.70%; recall:  72.13%; FB1:  74.34  867
              ORG: precision:  58.69%; recall:  61.45%; FB1:  60.04  1404
              PER: precision:  60.84%; recall:  63.25%; FB1:  62.02  1915


(69.50973807924782, 69.67351060249074, 69.59152798789712)

In [ ]:
samplenum = 2166

print(val_raw['text'][samplenum])
print(val_raw['tags'][samplenum])
print(prediction[samplenum])

In [29]:
# YOU SHOULD NOT CHANGE THIS CODEBLOCK
# make prediction on the test set and save to submission.txt
preds = predict(model, test_dataloader, device)
with open("submission.txt", "w") as f:
    for tags in preds:
        f.write(" ".join(tags) + "\n")

100%|██████████| 108/108 [00:01<00:00, 102.07it/s]


In [ ]:
pwd

In [ ]:
ls